### Install Geemap

In [1]:
!pip install geemap

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.3 MB 3.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.3 MB 4.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.3 MB 4.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.3 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.2 MB 16.4 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 2.6 MB/s eta 0:00:01
   --------------------------

### Import Geemap

In [1]:
import geemap

In [2]:
import ipyleaflet

### Verification for Earth Engine to Notebooks

In [3]:
m = ipyleaflet.Map()

In [4]:
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [5]:
my_map = geemap.Map()
my_map

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Import Earth Engine

In [6]:
import ee

### Import Area of interest

In [7]:
NMA = ee.FeatureCollection('projects/ee-maichmarc/assets/NMA')

### Import Training Data

In [8]:
Urban = ee.FeatureCollection('projects/ee-maichmarc/assets/Urban')
Forest = ee.FeatureCollection('projects/ee-maichmarc/assets/Forest')
Agriculture = ee.FeatureCollection('projects/ee-maichmarc/assets/Agriculture')
Water = ee.FeatureCollection('projects/ee-maichmarc/assets/Water')
Rangeland = ee.FeatureCollection('projects/ee-maichmarc/assets/Rangeland')

### Import landsat 8 data for 2018

In [9]:
l8_2018_dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterDate('2018-01-01', '2018-12-31')\
    .filterBounds(NMA)\
    .sort('CLOUD_COVER')\
    .first()

### Create visaualisation parameters

In [10]:
vis_params = {
  'min': 0,
  'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

### Add imported dataset to our Geemap map

In [11]:
my_map.addLayer(l8_2018_dataset, vis_params,'L8_2018')

In [12]:
my_map.centerObject(NMA)
my_map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [13]:
map_2018 = geemap.Map()

In [14]:
l8_2018_NMA = l8_2018_dataset.clip(NMA)

In [15]:
map_2018.addLayer(l8_2018_NMA, vis_params,'L8_NMA')
map_2018.centerObject(NMA)
map_2018

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Training parameters

In [16]:
label = 'Class'
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7'];
inputImage = l8_2018_NMA.select(bands);

In [17]:
training = Urban.merge(Forest).merge(Agriculture).merge(Water).merge(Rangeland)

In [18]:
trainImage = inputImage.sampleRegions(
    **{'collection': training, 'properties': [label], 'scale': 30}
)

### Split training data into training and validation sets

In [19]:
trainingData = trainImage.randomColumn()
split = 0.8
trainSet = trainingData.filter(ee.Filter.lessThan('random', split))
validationSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', split))

### Train the smileCART classifier

In [20]:
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)


### Classify the image

In [21]:
classified = inputImage.classify(classifier)

### Create a visualisation pallete for the classified image

In [22]:
LULC_palette = [
  'd63000', # Urban (1)
  '05450a', # Forest (2)
  'f096ff', # Agriculture (3)
  '0064c8', # Water (4)
  'ffff4c', # Rangeland (4)
  ]

### Add classified image to map

In [23]:
map_2018.centerObject(NMA)
map_2018.addLayer(classified, {'palette': LULC_palette, 'min': 1, 'max': 5},'LULC_2023')
map_2018

Map(bottom=132309.0, center=[-1.3026461017265567, 36.95923680466546], controls=(WidgetControl(options=['positi…

### Model validation

In [26]:
validated = validationSet.classify(classifier)

In [27]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_0000000000000000000c_0',
 'properties': {'Class': 1,
  'SR_B1': 9154,
  'SR_B2': 9694,
  'SR_B3': 10873,
  'SR_B4': 11667,
  'SR_B5': 14094,
  'SR_B7': 15045,
  'classification': 1,
  'random': 0.9478296975690637}}

In [26]:
test_accuracy = validated.errorMatrix('Class','classification')

In [27]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0],
 [0, 20, 0, 1, 0, 7],
 [0, 0, 21, 0, 0, 0],
 [0, 1, 0, 16, 0, 0],
 [0, 0, 1, 0, 19, 0],
 [0, 1, 0, 0, 0, 26]]

In [28]:
test_accuracy.accuracy().getInfo()

0.9026548672566371

In [29]:
test_accuracy.kappa().getInfo()

0.8773072747014115

In [30]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.7142857142857143],
 [1],
 [0.9411764705882353],
 [0.95],
 [0.9629629629629629]]

In [31]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9090909090909091,
  0.9545454545454546,
  0.9411764705882353,
  1,
  0.7878787878787878]]

### Export classified image

In [32]:
import os

In [33]:
cwd = os.getcwd() 
print(cwd)

C:\Users\MARK


In [34]:
filename_01 = os.path.join(cwd, 'NMA_LULC_2023.tif')

In [35]:
NMA_roi = NMA.geometry()

In [36]:
geemap.ee_export_image(classified, filename=filename_01, scale=90, region=NMA_roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\MARK\NMA_LULC_2023.tif


In [ ]:
DEM = ee.Image('projects/ee-maichmarc/assets/my_dem')

In [ ]:
slope = ee.Terrain.slope(DEM)
slopevis = {'min':0, 'max':35, 'palette': ['green', 'yellow', 'red']}

In [ ]:
DEM_map = geemap.Map()

In [ ]:
DEM_map.addLayer(DEM.clip(NMA), {'min':0, 'max':4000}, 'DEM')
DEM_map

In [ ]:
Slope_map = geemap.Map()

In [ ]:
Slope_map.addLayer(slope.clip(NMA), slopevis, 'slope')
Slope_map

In [ ]:
filename_DEM = os.path.join(cwd, 'DEM_NMA.tif')

In [ ]:
geemap.ee_export_image(DEM.clip(NMA), filename=filename_DEM, scale=60, region=NMA_roi, file_per_band=True)

In [ ]:
filename_slope = os.path.join(cwd, 'slope_NMA.tif')

In [ ]:
geemap.ee_export_image(slope.clip(NMA), filename=filename_slope, scale=60, region=NMA_roi, file_per_band=True)

In [ ]:
geemap.ee_export_image(slope.clip(NMA), filename=filename_slope, scale=60, region=NMA_roi, file_per_band=True)

In [ ]:
filename_slope = os.path.join(cwd, 'NMA_slope.tif')

In [ ]:
geemap.ee_export_image(DEM.clip(NMA), filename=filename_DEM, scale=60, region=NMA_roi, file_per_band=True)

In [ ]:
filename_DEM = os.path.join(cwd, 'NMA_DEM.tif')

#### Export as tif files

In [ ]:
Slope_map.addLayer(slope.clip(NMA), slopevis, 'slope')
Slope_map

In [ ]:
Slope_map = geemap.Map()

In [ ]:
DEM_map.addLayer(DEM.clip(NMA), {'min':0, 'max':4000}, 'DEM')
DEM_map

In [ ]:
DEM_map = geemap.Map()

#### Dispaly both maps

In [ ]:
slope = ee.Terrain.slope(DEM)
slopevis = {'min':0, 'max':35, 'palette': ['green', 'yellow', 'red']}

#### Create slope image from DEM tif file

In [ ]:
DEM = ee.Image('projects/ee-maichmarc/assets/my_dem')

#### Import DEM tif file from Earth Engine

### Prepare DEM map and Slope map for QGIS prediction

In [ ]:
geemap.ee_export_image(NMA_Modis_2021, filename=filename_21_1, scale=60, region=NMA_roi, file_per_band=True)

In [ ]:
filename_21_1 = os.path.join(cwd, 'NMA_2021_1.tif')